## Set Up Dependencies and Data


In [ ]:
import joblib
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm


In [ ]:
slip_df = pd.read_csv("https://osf.io/s36qy/download")
baseline_df = pd.read_csv("https://osf.io/a48ch/download")


## Reproducibility


In [ ]:
%load_ext watermark
%watermark -iwbmuvg -iv


In [ ]:
joblib.hash(slip_df)


In [ ]:
joblib.hash(baseline_df)


# Preprocess data


In [ ]:
slip_df["condition"] = "slip"
baseline_df["condition"] = "baseline"

df = pd.concat([slip_df, baseline_df])


In [ ]:
df["Task Coding Sites"] = df["Task Coding Sites"].apply(eval)


In [ ]:
df["Run ID"] = df["Unnamed: 0"].apply(lambda x: x.split(",")[0])
df["Task"] = df["Unnamed: 0"].apply(lambda x: x.split(",")[1])


In [ ]:
tidy_df = df.explode("Task Coding Sites")
tidy_df


### Draw Genome Maps


In [ ]:
n_groups = len(tidy_df.groupby(["Run ID", "Length", "condition"]))
n_rows = int(np.ceil(np.sqrt(n_groups)))
n_cols = n_rows
plt.figure(figsize=(n_cols * 10, n_rows * 6))

for ax, (idx, group_df) in tqdm(
    zip(
        plt.subplots(
            n_rows,
            n_cols,
            sharex=True,
            sharey=True,
            squeeze=False,
            figsize=(n_cols * 10, n_rows * 6),
        )[1].ravel(),
        tidy_df.groupby(["Run ID", "Length", "condition"], sort=True),
    ),
):

    pivot_df = group_df.pivot_table(
        aggfunc="count",
        columns="Task Coding Sites",
        index="Task",
        values="Length",
    )
    for col in pivot_df.columns:
        vals = pivot_df[col].values
        pivot_df[col] = pivot_df[col] * np.array(range(len(vals)))

    for task in df["Task"].unique():
        if task not in pivot_df.index:
            pivot_df.loc[task] = np.nan

    df.sort_index(axis=1)

    sns.heatmap(
        pivot_df,
        ax=ax,
        cbar=False,
        cmap=sns.color_palette()[:10],
        vmin=0,
        vmax=10,
    )
    ax.set_title(idx[2])


plt.savefig("genome_layouts.png", dpi=500)
